# Fundamental 27. SQL 

## Python DB API
파이썬에서 DB 연결을 위한 전화선인 sqlite3 모듈을 사용한다.

In [2]:
import sqlite3
import os
db_path = os.getenv('HOME')+'/mydb.db'
conn = sqlite3.connect(db_path)  # mydb.db에 연결합니다.
print(conn)

sqlite3 모듈은 파이썬 표준 라이브러리로 SQLite에 대한 인터페이스를 기본적으로 제공한다.

`conn` 객체에는 SQL 연결과 관련된 셋팅이 포함되어 있다. 이번에는 `Connect()` 함수의 연결을 사용하는 새로운 Cursor 객체를 만들자.

In [3]:
c = conn.cursor()
print(c)

Cursor는 SQL 질의(Query)를 수행하고 결과를 얻는데 사용하는 객체이다. `INSERT`처럼 DB에만 적용되는 명령어를 사용한다면 Cursor를 안 사용할 수 있지만 SELECT와 같이 데이터를 불러올 때는 SQL 질의 수행 결과에 접근하기 위한 Cursor가 반드시 필요하다.

In [4]:
# stocks이라는 이름의 테이블을 하나 생성합니다. 혹시 이미 생성되었다면 생략합니다. 
c.execute("CREATE TABLE  IF NOT EXISTS  stocks (date text, trans text, symbol text, qty real, price real)")

# stocks 테이블에 데이터를 하나 인서트합니다. 
c.execute("INSERT INTO stocks VALUES ('20200701', 'TEST', 'AIFFEL', 1, 10000)")

# 방금 인서트한 데이터를 조회해 봅니다.
c.execute("SELECT * FROM stocks")

# 조회된 내역을 커서를 통해 가져와 출력해 봅니다.
print(c.fetchone())

('20200701', 'TEST', 'AIFFEL', 1.0, 10000.0)


### commit

삽입, 갱신, 삭제 등의 SQL 질의가 끝났다면 `conn.commit()`를 호출해야 DB가 실제로 업데이트 된다.

In [5]:
conn.commit()   # commit()은 cursor의 메소드가 아니라 connection의 메소드

commit()을 통해 데이터베이스에 데이터 변경이 실제적으로 반영되었다.  
이런 것을 데이터베이스에서는 __트랜잭션(transaction) 관리__라고 한다.

### close

commit()을 완료했다면 DB와 대화하는 것을 마무리 지어야한다.  
`close()` 를 이용 대화의 마무리는 DB와의 연결을 끊는 것으로 이루어진다. 

In [6]:
c.close()      # 먼저 커서를 닫은 후
conn.close()    # DB 연결을 닫아 줍니다.

### DDL 문으로 테이블 생성

In [7]:
import sqlite3
import os
db_path = os.getenv('HOME')+'/mydb.db'

conn = sqlite3.connect(db_path)
c = conn.cursor()

#- ! 재실행 시 테이블이 존재할 수 있으므로 아래처럼 해당 테이블들을 모두 지워줍니다.
c.execute("DROP TABLE IF EXISTS 도서대출내역")
c.execute("DROP TABLE IF EXISTS 도서대출내역2")
c.execute("DROP TABLE IF EXISTS 대출내역")
c.execute("DROP TABLE IF EXISTS 도서명")


#----- 1st table : 도서대출내역 -----#
c.execute("CREATE TABLE IF NOT EXISTS 도서대출내역 (ID varchar, 이름 varchar, 도서ID varchar, 대출일 varchar, 반납일 varchar)")
#- 생성(create)문 : 테이블명, 변수명, 변수타입을 지정

data = [('101','문강태','aaa','2020-06-01','2020-06-05'),
             ('101','문강태','ccc','2020-06-20','2020-06-25'),
             ('102','고문영','bbb','2020-06-01',None),
             ('102','고문영','ddd','2020-06-08',None),
             ('103','문상태','ccc','2020-06-01','2020-06-05'),
             ('104','강기둥',None,None,None)]
#- 입력할 데이터를 그대로 입력 (변수명 순서 기준대로)

c.executemany('INSERT INTO 도서대출내역 VALUES (?,?,?,?,?)', data)
#- 입력할 데이터를 실제 테이블에 insert하기
#-----------------------------------------------#


#----- 2nd table : 도서대출내역2 -----#
c.execute("CREATE TABLE IF NOT EXISTS 도서대출내역2 (ID varchar, 이름 varchar, 대출년월 varchar, 대출일수 varchar)")

data = [('101','문강태','2020-06','20일'),
             ('102','고문영','2020-06','10일'),
             ('103','문상태','2020-06','8일'),
             ('104','강기둥','2020-06','3일')]
c.executemany('INSERT INTO 도서대출내역2 VALUES (?,?,?,?)', data)
#--------------------------------------------------#


#----- 3rd table : 대출내역 -----#
c.execute("CREATE TABLE IF NOT EXISTS 대출내역 (ID varchar, 이름 varchar, 도서ID varchar)")

data = [('101','문강태','aaa'),
             ('102','고문영','bbb'),
             ('102','고문영','fff'),
             ('103','문상태','ccc'),
             ('104','강기둥',None)]
c.executemany('INSERT INTO 대출내역 VALUES (?,?,?)', data)
#-----------------------------------------#


#----- 4th table : 도서명 -----#
c.execute("CREATE TABLE IF NOT EXISTS 도서명 (도서ID varchar, 도서명 varchar)")

data = [('aaa','악몽을 먹고 자란 소년'),
             ('bbb','좀비아이'),
             ('ccc','공룡백과사전'),
             ('ddd','빨간구두'),
             ('eee','잠자는 숲속의 미녀')]

c.executemany('INSERT INTO 도서명 VALUES (?,?)', data)
#--------------------------------------#

conn.commit()
conn.close()

데이터의 생성/삭제/갱신 등을 처리할 때 필요한 DML문의 호출 방법에 대해 바로 위에서 다루었던 코드를 통해 이해해보자.  
```python
#----- 4th table : 도서명 -----#

# [1]
c.execute("CREATE TABLE IF NOT EXISTS 도서명 (도서ID varchar, 도서명 varchar)")

# [2]
data = [('aaa','악몽을 먹고 자란 소년'),
             ('bbb','좀비아이'),
             ('ccc','공룡백과사전'),
             ('ddd','빨간구두'),
             ('eee','잠자는 숲속의 미녀')]

# [3]
c.executemany('INSERT INTO 도서명 VALUES (?,?)', data)
#--------------------------------------#
```


- [1] `CREATE TABLE IF NOT EXISTS 도서명 (도서ID varchar, 도서명 varchar)`
  - 만약 테이블이 존재하지 않으면 테이블을 생성해 주세요. 그리고 변수명과 각 변수의 타입은 다음과 같이 설정해 주세요!"라는 의미
  

- [2] 각 변수명에 맞게 데이터를 실제로 생성
  

- [3] `INSERT INTO 도서명 VALES (?, ?)`
  - 각 테이블의 변수(도서ID, 도서명)에 2번에서 입력한 데이터(data)를 넣겠다는 뜻
  - `?`를 사용해서 데이터를 쿼리에 바인딩하는 기법을 매우 흔히 사용
  - `c.executemany()` 는 한꺼번에 여러 개의 데이터 처리를 가능하게 해줌. 입력해야 할 데이터를 위와 같이 list로 관리하고 있는 경우 매우 유용

In [9]:
conn = sqlite3.connect(db_path)
c = conn.cursor()

for row in c.execute('SELECT * FROM 도서명'):
  print(row)

('aaa', '악몽을 먹고 자란 소년')
('bbb', '좀비아이')
('ccc', '공룡백과사전')
('ddd', '빨간구두')
('eee', '잠자는 숲속의 미녀')


-----
# SQL 기본

![image](https://user-images.githubusercontent.com/84179578/132537235-d55e8a99-36cc-459f-b381-92f3052f816a.png)

In [10]:
#- 참고 : 실제 조회를 해보시려면 아래처럼 쓰시면 됩니다.
#- c.execute() 괄호 안에 SQL문을 넣으시면 됩니다.

import os
db_path = os.getenv('HOME')+'/mydb.db'
conn = sqlite3.connect(db_path)  # mydb.db에 연결합니다.
c = conn.cursor()

for row in c.execute('SELECT * FROM 도서대출내역'):
  print(row)

#- ! 출력 시 'NULL' 대신 'None'으로 출력될 수 있으나 동일하게 이해하시면 됩니다.

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('101', '문강태', 'ccc', '2020-06-20', '2020-06-25')
('102', '고문영', 'bbb', '2020-06-01', None)
('102', '고문영', 'ddd', '2020-06-08', None)
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')
('104', '강기둥', None, None, None)


### 전체 조회

In [11]:
for row in c.execute('SELECT * FROM 도서대출내역'):
  print(row)

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('101', '문강태', 'ccc', '2020-06-20', '2020-06-25')
('102', '고문영', 'bbb', '2020-06-01', None)
('102', '고문영', 'ddd', '2020-06-08', None)
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')
('104', '강기둥', None, None, None)


### 특정 컬럼을 지정

In [12]:
for row in c.execute('SELECT ID FROM 도서대출내역'):
  print(row)

('101',)
('101',)
('102',)
('102',)
('103',)
('104',)


### 조건을 입력

In [13]:
for row in c.execute('SELECT * FROM 도서대출내역 WHERE 이름 = "문강태";'):
  print(row)

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('101', '문강태', 'ccc', '2020-06-20', '2020-06-25')


### GROUP BY로 중복을 제거

In [14]:
for row in c.execute('SELECT 이름 FROM 도서대출내역 GROUP BY 이름;'):
  print(row)

('강기둥',)
('고문영',)
('문강태',)
('문상태',)


### DISTINCT로 중복을 제거

In [15]:
for row in c.execute('SELECT DISTINCT 이름 FROM 도서대출내역;'):
  print(row)

('문강태',)
('고문영',)
('문상태',)
('강기둥',)


### ORDER BY로 정렬

오름차순

In [16]:
for row in c.execute('SELECT * FROM 도서대출내역 ORDER BY ID ;'):
  print(row)

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('101', '문강태', 'ccc', '2020-06-20', '2020-06-25')
('102', '고문영', 'bbb', '2020-06-01', None)
('102', '고문영', 'ddd', '2020-06-08', None)
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')
('104', '강기둥', None, None, None)


내림차순

In [17]:
for row in c.execute('SELECT * FROM 도서대출내역 ORDER BY ID DESC ;'):
  print(row)

('104', '강기둥', None, None, None)
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')
('102', '고문영', 'bbb', '2020-06-01', None)
('102', '고문영', 'ddd', '2020-06-08', None)
('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('101', '문강태', 'ccc', '2020-06-20', '2020-06-25')


### 몇개의 row만 조회

In [18]:
for row in c.execute('SELECT * FROM 도서대출내역 LIMIT 5 ;'):
  print(row)

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('101', '문강태', 'ccc', '2020-06-20', '2020-06-25')
('102', '고문영', 'bbb', '2020-06-01', None)
('102', '고문영', 'ddd', '2020-06-08', None)
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')


In [19]:
for row in c.execute('SELECT 이름, 대출일, 반납일 FROM 도서대출내역 ORDER BY 대출일 DESC LIMIT 1;'):
  print(row)

('문강태', '2020-06-20', '2020-06-25')


## 데이터 타입
데이터의 타입(형)을 바꾸는 실습을 해보자.

In [20]:
for row in c.execute('SELECT * FROM 도서대출내역2;'):
  print(row)

('101', '문강태', '2020-06', '20일')
('102', '고문영', '2020-06', '10일')
('103', '문상태', '2020-06', '8일')
('104', '강기둥', '2020-06', '3일')


In [21]:
import os
db_path = os.getenv('HOME')+'/mydb.db'
conn = sqlite3.connect(db_path)  # mydb.db에 연결합니다.
c = conn.cursor()

for row in c.execute('pragma table_info(도서대출내역)'):
  print(row)

(0, 'ID', 'varchar', 0, None, 0)
(1, '이름', 'varchar', 0, None, 0)
(2, '도서ID', 'varchar', 0, None, 0)
(3, '대출일', 'varchar', 0, None, 0)
(4, '반납일', 'varchar', 0, None, 0)


데이터 타입을 보니 모두 `VARCHAR` , 문자형 이다.

문자형의 특정 부분을 떼어 내는 함수는 `LEFT`, `RIGHT`, `SUBSTRING`를 사용

In [23]:
for row in c.execute('SELECT *, SUBSTR(대출일수, 1, (length(대출일수)-1)) AS 대출일수_수정 FROM 도서대출내역2;'):
  print(row)

('101', '문강태', '2020-06', '20일', '20')
('102', '고문영', '2020-06', '10일', '10')
('103', '문상태', '2020-06', '8일', '8')
('104', '강기둥', '2020-06', '3일', '3')


`CAST` 함수를 이용하여, 잘라낸 부분에 더하여 숫자로 변환

In [24]:
for row in c.execute('SELECT *, CAST(SUBSTR(대출일수, 1, (length(대출일수)-1)) AS INT) AS 대출일수_수정 FROM 도서대출내역2 ;'):
  print(row)

('101', '문강태', '2020-06', '20일', 20)
('102', '고문영', '2020-06', '10일', 10)
('103', '문상태', '2020-06', '8일', 8)
('104', '강기둥', '2020-06', '3일', 3)


집계함수와 `GROUP BY` 를 사용하여 평균 구하기

In [25]:
for row in c.execute('SELECT ID, 이름, 대출년월, AVG(CAST(SUBSTR(대출일수, 1, (length(대출일수)-1)) AS INT)) AS 대출일수_평균 FROM 도서대출내역2 GROUP BY 1,2,3;'):
  print(row)

('101', '문강태', '2020-06', 20.0)
('102', '고문영', '2020-06', 10.0)
('103', '문상태', '2020-06', 8.0)
('104', '강기둥', '2020-06', 3.0)


## 다양한 조건으로 조회

### WHERE 조건에 다양한 조건 입력
#### 1. 특정 문자를 포함하는 row를 가져오고 싶을 때

In [26]:
for row in c.execute('SELECT * FROM 도서대출내역2 WHERE 이름 LIKE "문%" ;'):
  print(row)

('101', '문강태', '2020-06', '20일')
('103', '문상태', '2020-06', '8일')


#### 2. 특정 기간 혹은 특정 날짜의 이전 또는 이후의 row를 가져오고 싶을 때

In [27]:
for row in c.execute('SELECT * FROM 도서대출내역 WHERE 대출일 >= "2020-06-01" AND 대출일 <= "2020-06-07" ;'):
  print(row)

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('102', '고문영', 'bbb', '2020-06-01', None)
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')


위 코드처럼 간단히 부등호를 사용해 결과를 조회할 수도 있고, BETWEEN이라는 함수를 활용할 수도 있다.

In [28]:
for row in c.execute('SELECT * FROM 도서대출내역 WHERE 대출일 BETWEEN "2020-06-01" AND "2020-06-07" ;'):
  print(row)

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('102', '고문영', 'bbb', '2020-06-01', None)
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')


BETWEEN의 경우에는 BETWEEN(시작일 AND 종료일)로 범위를 정해 사용할 수 있다.

#### 3.특정 숫자 이상 또는 이하의 row를 조회하고 싶을 때

In [29]:
for row in c.execute('SELECT *, CAST(SUBSTR(대출일수, 1, (length(대출일수)-1)) AS INT) AS 대출일수_수정 FROM 도서대출내역2 WHERE 대출일수_수정 > 5 '):
  print(row)

('101', '문강태', '2020-06', '20일', 20)
('102', '고문영', '2020-06', '10일', 10)
('103', '문상태', '2020-06', '8일', 8)


#### 4.NULL 조건을 다루는 방법

In [30]:
for row in c.execute('SELECT * FROM 도서대출내역 WHERE 반납일 IS NOT NULL;'):
  print(row)

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('101', '문강태', 'ccc', '2020-06-20', '2020-06-25')
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')


In [31]:
for row in c.execute('SELECT * FROM 도서대출내역 WHERE 반납일 IS NULL;'):
  print(row)

('102', '고문영', 'bbb', '2020-06-01', None)
('102', '고문영', 'ddd', '2020-06-08', None)
('104', '강기둥', None, None, None)


----
## JOIN

In [32]:
import os
db_path = os.getenv('HOME')+'/mydb.db'
conn = sqlite3.connect(db_path)  # mydb.db에 연결합니다.
c = conn.cursor()

In [33]:
print('대출내역 테이블')
for row in c.execute('SELECT * FROM 대출내역;'):
  print(row)
print('')
print('도서명 테이블')
for row in c.execute('SELECT * FROM 도서명;'):
  print(row)

대출내역 테이블
('101', '문강태', 'aaa')
('102', '고문영', 'bbb')
('102', '고문영', 'fff')
('103', '문상태', 'ccc')
('104', '강기둥', None)

도서명 테이블
('aaa', '악몽을 먹고 자란 소년')
('bbb', '좀비아이')
('ccc', '공룡백과사전')
('ddd', '빨간구두')
('eee', '잠자는 숲속의 미녀')


### INNER JOIN

In [34]:
query = '''
SELECT A.*, B.도서명 
FROM 대출내역 AS A 
INNER JOIN 도서명 AS B
ON A.도서ID = B.도서ID;
'''
for row in c.execute(query):
  print(row)

('101', '문강태', 'aaa', '악몽을 먹고 자란 소년')
('102', '고문영', 'bbb', '좀비아이')
('103', '문상태', 'ccc', '공룡백과사전')


### LEFT JOIN

In [35]:
query = '''
SELECT A.*, B.도서명 
FROM 대출내역 AS A 
LEFT JOIN 도서명 AS B
ON A.도서ID = B.도서ID;
'''
for row in c.execute(query):
  print(row)

('101', '문강태', 'aaa', '악몽을 먹고 자란 소년')
('102', '고문영', 'bbb', '좀비아이')
('102', '고문영', 'fff', None)
('103', '문상태', 'ccc', '공룡백과사전')
('104', '강기둥', None, None)


### RIGHT JOIN

In [36]:
query = '''
SELECT B.*, A.ID, A.이름 
FROM 대출내역 AS A 
RIGHT JOIN 도서명 AS B
ON A.도서ID = B.도서ID;
'''
for row in c.execute(query):
  print(row)

OperationalError: RIGHT and FULL OUTER JOINs are not currently supported

지금은 에러가 나지만 만약 다른 RDBMS 프로그램을 사용한다면 오른쪽 테이블을 기준으로 해서 왼쪽 테이블 중 공통인 것만 붙고 나머지는 전부 NULL처리가 되었을것이다.

### FULL OUTER JOIN

In [37]:
query = '''
SELECT A.*, B.도서명 
FROM 대출내역 AS A 
FULL OUTER JOIN 도서명 AS B
ON A.도서ID = B.도서ID
ORDER BY 도서ID;
'''
for row in c.execute(query):
  print(row)

OperationalError: RIGHT and FULL OUTER JOINs are not currently supported

지금은 에러가 나지만 만약 다른 RDBMS 프로그램을 사용한다면 도서ID를 기준으로 붙을 수 있는 것은 모두 붙고 나머지는 NULL 값이 좌우로 붙은 것을 확인할 수 있다.

### 중첩 질의(Nested Query)

In [38]:
query = '''
SELECT C.이름, COUNT(*) 대출건수
FROM (
	SELECT A.*, B.도서명 
	FROM 대출내역 AS A 
	LEFT JOIN 도서명 AS B
	ON A.도서ID = B.도서ID ) C
GROUP BY C.이름;
'''
for row in c.execute(query):
  print(row)

('강기둥', 1)
('고문영', 2)
('문강태', 1)
('문상태', 1)


### 쿼리의 조건절(1) IFNULL

In [39]:
query = '''
SELECT A.*, IFNULL(B.도서명, '도서명미상') AS 도서명
	FROM 대출내역 AS A 
	LEFT JOIN 도서명 AS B
	ON A.도서ID = B.도서ID;
'''
for row in c.execute(query):
  print(row)

('101', '문강태', 'aaa', '악몽을 먹고 자란 소년')
('102', '고문영', 'bbb', '좀비아이')
('102', '고문영', 'fff', '도서명미상')
('103', '문상태', 'ccc', '공룡백과사전')
('104', '강기둥', None, '도서명미상')


 NULL 대신 다른 값으로 바꾸어 출력하고 싶을때 사용되는 것이 `IFNULL(값1, 값2)` 함수 이다.

### 쿼리의 조건절(2) CASE

In [40]:
query = '''
SELECT 이름, 
CASE WHEN 대출일수_수정 > 5 
    THEN '기간초과' 
    ELSE '기간내' 
END AS 대출기간
FROM (
    SELECT *, CAST(SUBSTR(대출일수, 1, (length(대출일수)-1)) AS INT) AS 대출일수_수정 
    FROM 도서대출내역2
);
'''
for row in c.execute(query):
  print(row)

('문강태', '기간초과')
('고문영', '기간초과')
('문상태', '기간초과')
('강기둥', '기간내')


CASE문은 마치 프로그래밍에서의 IF문처럼 조건에 따라 다양한 출력이 가능하도록 해주는 매우 강력하고 유용한 함수이다.